# CatBoost Machine Learning

Prediction of Next Sales Revenue

In [1]:
#Core Libraries for Data Explorer
import pandas as pd

#Scientific computing
import numpy as np

# Modelling
from catboost import CatBoostRegressor

# Evaluation
from sklearn.metrics import mean_absolute_error as mae 

#Ignore Some Unuseful Warnings
import warnings
warnings.filterwarnings("ignore")

# Data Exploration

* Importing Data Set from DataSets Folder

In [2]:
df_train = pd.read_csv("DataSets/train.csv")

df_train

,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count
0,merchant_43992,202307,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,15106
1,merchant_43992,202301,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,16918
2,merchant_43992,202305,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,13452
3,merchant_43992,202308,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,16787
4,merchant_43992,202302,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,12428
...,...,...,...,...,...,...,...,...
291137,merchant_67083,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_169,Segment - 4,4
291138,merchant_4263,202309,Merchant Source - 2,Settlement Period - 1,Working Type - 6,mcc_73,Segment - 4,4
291139,merchant_11346,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_169,Segment - 4,3
291140,merchant_21397,202309,Merchant Source - 1,Settlement Period - 1,Working Type - 6,mcc_168,Segment - 4,3


This function uniqe for FinPy. It will describe data set statues for cleaning progress. 

In [3]:
def summary(self):
    start_mememory = self.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB \n ---'.format(start_mememory))
    
    row = self.shape[0]
    print(f"Total of Rows within one column: {row} \n ---")
    
    column = self.shape[1]
    print(f"Total of Columns: {column} \n ---")

    
    self_null = self.isnull().sum().sum()     #Sum of Columns inculidng Nan
    print(f"Nan Values Including total of Column: {self_null} \n ---")

    Rows_duplicate = len(self)-len(self.drop_duplicates()) #count number of duplicate in Dataframe

    print(f"All Rows Duplicate Count: {Rows_duplicate} \n ---")

    columnnames = pd.DataFrame(self.dtypes).reset_index()
    numerical_column_list_float = columnnames[columnnames[0] == "float64"]["index"].to_list()
    numerical_column_list_int = columnnames[columnnames[0] == "int64"]["index"].to_list()
    numerical_column_list = numerical_column_list_float
    numerical_column_list.extend(numerical_column_list_int)
    
    sumofnumerical_columns_list = len(numerical_column_list_float) + len(numerical_column_list_int)
    string_column_list = columnnames[columnnames[0] == "object"]["index"].to_list()
    sumofstring_column_list = len(string_column_list)

    print(f"Numerical Column Names {numerical_column_list} \n ---")
    print(f"Sum Of numerical columns are {sumofnumerical_columns_list} \n ---")
    print(f"String Column Names {string_column_list} \n ---")
    print(f"Sum of string columns are {sumofstring_column_list} \n ---")

    zero = (self == 0.00).astype(int).sum(axis = 0).sum()

    print(f"All zero numbers total is {zero}")

In [4]:
summary(df_train)

Memory usage of dataframe is 17.77 MB 
 ---
Total of Rows within one column: 291142 
 ---
Total of Columns: 8 
 ---
Nan Values Including total of Column: 0 
 ---
All Rows Duplicate Count: 0 
 ---
Numerical Column Names ['month_id', 'net_payment_count'] 
 ---
Sum Of numerical columns are 4 
 ---
String Column Names ['merchant_id', 'merchant_source_name', 'settlement_period', 'working_type', 'mcc_id', 'merchant_segment'] 
 ---
Sum of string columns are 6 
 ---
All zero numbers total is 72


Machine is understand numerical values. So, this process is converting string to int or float. 

In [5]:
def string_to_intiger(name):
    
    id = df_train[name].value_counts()
    id = id.index.to_list()

    # Create a mapping dictionary
    string_to_integer_id = {string: index + 1 for index, string in enumerate(id)}

    id_df = df_train[[name]]

    # Apply the mapping to the pd.Series
    df_train[name] = id_df[name].map(string_to_integer_id)

    return df_train

columns_to_string = ["merchant_id","merchant_source_name","settlement_period","working_type","mcc_id","merchant_segment"]

for i in columns_to_string:
    new_df = string_to_intiger(i)

In [6]:
new_df

,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count
0,1,202307,3,2,3,117,2,15106
1,1,202301,3,2,3,117,2,16918
2,1,202305,3,2,3,117,2,13452
3,1,202308,3,2,3,117,2,16787
4,1,202302,3,2,3,117,2,12428
...,...,...,...,...,...,...,...,...
291137,21981,202309,1,1,1,21,1,4
291138,21980,202309,2,1,1,11,1,4
291139,21979,202309,1,1,1,21,1,3
291140,21978,202309,1,1,1,8,1,3


In [7]:
df_train = df_train[df_train["net_payment_count"] != 0]

df_train

,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count
0,1,202307,3,2,3,117,2,15106
1,1,202301,3,2,3,117,2,16918
2,1,202305,3,2,3,117,2,13452
3,1,202308,3,2,3,117,2,16787
4,1,202302,3,2,3,117,2,12428
...,...,...,...,...,...,...,...,...
291137,21981,202309,1,1,1,21,1,4
291138,21980,202309,2,1,1,11,1,4
291139,21979,202309,1,1,1,21,1,3
291140,21978,202309,1,1,1,8,1,3


In [9]:
df_train['date'] = pd.to_datetime(df_train['month_id'].apply(lambda x: f'{str(x)[:4]}/{str(x)[4:]}/01')) # fixing date format 

df_train["Year"] = pd.to_datetime(df_train["date"]).dt.year

df_train["Month"] = pd.to_datetime(df_train["date"]).dt.month

df_train


,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count,date,Year,Month
0,1,202307,3,2,3,117,2,15106,2023-07-01,2023,7
1,1,202301,3,2,3,117,2,16918,2023-01-01,2023,1
2,1,202305,3,2,3,117,2,13452,2023-05-01,2023,5
3,1,202308,3,2,3,117,2,16787,2023-08-01,2023,8
4,1,202302,3,2,3,117,2,12428,2023-02-01,2023,2
...,...,...,...,...,...,...,...,...,...,...,...
291137,21981,202309,1,1,1,21,1,4,2023-09-01,2023,9
291138,21980,202309,2,1,1,11,1,4,2023-09-01,2023,9
291139,21979,202309,1,1,1,21,1,3,2023-09-01,2023,9
291140,21978,202309,1,1,1,8,1,3,2023-09-01,2023,9


In [10]:
# Creating New Dataframe for Machine Learning Algorithm
X = df_train.iloc[:,:] #.drop("month_id", axis=1)  # X is numerical Value

#X = X.drop("merchant_id",axis=1)

y = df_train["net_payment_count"]
display(y)

X = X.drop("net_payment_count",axis=1)
X = X.drop("date",axis=1)
display(X)

0         15106
1         16918
2         13452
3         16787
4         12428
          ...  
291137        4
291138        4
291139        3
291140        3
291141        3
Name: net_payment_count, Length: 291070, dtype: int64

,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,Year,Month
0,1,202307,3,2,3,117,2,2023,7
1,1,202301,3,2,3,117,2,2023,1
2,1,202305,3,2,3,117,2,2023,5
3,1,202308,3,2,3,117,2,2023,8
4,1,202302,3,2,3,117,2,2023,2
...,...,...,...,...,...,...,...,...,...
291137,21981,202309,1,1,1,21,1,2023,9
291138,21980,202309,2,1,1,11,1,2023,9
291139,21979,202309,1,1,1,21,1,2023,9
291140,21978,202309,1,1,1,8,1,2023,9


# Outside Data

Outside date will help our real data set for good prediction. More and relevant data sets are impact of our algorithm prediction and forecast. 

In [11]:
import yfinance as yf
tl_dolar = yf.download("TRY=X")
tl_close_dolar = tl_dolar[["Close"]].reset_index()
tl_close_dolar = tl_close_dolar[(tl_close_dolar["Date"] >= "2020-01-01") & (tl_close_dolar["Date"] <= "2023-12-31")]
tl_close_dolar["Month"] = tl_close_dolar["Date"].dt.month
tl_close_dolar["Year"]  = tl_close_dolar["Date"].dt.year

tl_mean_dolar = tl_close_dolar.groupby([tl_close_dolar["Month"],tl_close_dolar["Year"]])["Close"].mean()
tl_mean_dolar = pd.DataFrame(tl_mean_dolar)

df_dolar = tl_mean_dolar.copy()

df_dolar = df_dolar.reset_index(level=["Month","Year"])


tl_eur = yf.download("EURTRY=X")
tl_close_eur = tl_eur[["Close"]].reset_index()
tl_close_eur = tl_close_eur[(tl_close_eur["Date"] >= "2020-01-01") & (tl_close_eur["Date"] <= "2023-12-31")]
tl_close_eur["Month"] = tl_close_eur["Date"].dt.month
tl_close_eur["Year"]  = tl_close_eur["Date"].dt.year

tl_mean_eur = tl_close_eur.groupby([tl_close_eur["Month"],tl_close_eur["Year"]])["Close"].mean()
tl_mean_eur = pd.DataFrame(tl_mean_eur)

df_eur = tl_mean_eur.copy()

df_eur = df_eur.reset_index(level=["Month","Year"])


tl_gbp = yf.download("GBPTRY=X")
tl_close_gbp = tl_gbp[["Close"]].reset_index()
tl_close_gbp = tl_close_gbp[(tl_close_gbp["Date"] >= "2020-01-01") & (tl_close_gbp["Date"] <= "2023-12-31")]
tl_close_gbp["Month"] = tl_close_gbp["Date"].dt.month
tl_close_gbp["Year"]  = tl_close_gbp["Date"].dt.year

tl_mean_gbp = tl_close_gbp.groupby([tl_close_gbp["Month"],tl_close_gbp["Year"]])["Close"].mean()
tl_mean_gbp = pd.DataFrame(tl_mean_gbp)

df_gbp = tl_mean_gbp.copy()

df_gbp = df_gbp.reset_index(level=["Month","Year"])

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [13]:
fed = pd.read_excel("DataSets\FEDFUNDS.xlsx")

fed["Month"] = fed["date"].dt.month
fed["Year"]  = fed["date"].dt.year

fed.head()

,date,FEDFUNDS,Month,Year
0,2020-01-01,1.55,1,2020
1,2020-02-01,1.58,2,2020
2,2020-03-01,0.65,3,2020
3,2020-04-01,0.05,4,2020
4,2020-05-01,0.05,5,2020


# Main Data

* Merging all useful data sets to one data set

In [14]:
merged_df = pd.merge(X, df_dolar, on=['Year',"Month"], how='inner')
merged_df.rename(columns={"Close":"Close-Dolar"}, inplace= True)
merged_df_2 = pd.merge(merged_df, df_eur, on=['Year',"Month"], how='inner')
merged_df_2.rename(columns={"Close":"Close-Eur"}, inplace= True)
merged_df_3 = pd.merge(merged_df_2, df_gbp, on=['Year',"Month"], how='inner')
merged_df_3.rename(columns={"Close":"Close-Gbp"}, inplace= True)
merged_df_4 = pd.merge(merged_df_3, fed, on=['Year',"Month"], how='inner')
merged_df_4.rename(columns={"Close":"FED"}, inplace= True)
merged_df_4 = merged_df_4.drop("date",axis=1)
merged_df_4

,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,Year,Month,Close-Dolar,Close-Eur,Close-Gbp,FEDFUNDS
0,1,202307,3,2,3,117,2,2023,7,26.419481,29.194770,34.004316,5.12
1,1,202301,3,2,3,117,2,2023,1,18.765020,20.216952,22.947173,4.33
2,1,202305,3,2,3,117,2,2023,5,19.726417,21.452771,24.622601,5.06
3,1,202308,3,2,3,117,2,2023,8,26.900117,29.320106,34.148310,5.33
4,1,202302,3,2,3,117,2,2023,2,18.843263,20.195404,22.786362,4.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
291065,21981,202309,1,1,1,21,1,2023,9,26.962478,28.759306,33.401534,5.33
291066,21980,202309,2,1,1,11,1,2023,9,26.962478,28.759306,33.401534,5.33
291067,21979,202309,1,1,1,21,1,2023,9,26.962478,28.759306,33.401534,5.33
291068,21978,202309,1,1,1,8,1,2023,9,26.962478,28.759306,33.401534,5.33


In [15]:
sep = int(len(X)*0.75)
sep

218302

Before the Machine Learning process, we have to seperate our data set Train and Test. 

Why we are making these split?
we are not now to future data prediction 100%. But we know the past data. We are decading past data for past date prediction. After that, ıf the parameters are perfect of algorithm. We are using these parameters for future prediction. It will increasing our efficency.  

In [16]:
X_train  = merged_df_4.iloc[:sep]
print(f" Shape of X_train : {X_train.shape}")
X_test = merged_df_4.iloc[sep:]
print(f" Shape of X_test : {X_test.shape}")
y_train  = y[:sep]
print(f" Shape of y_train : {y_train.shape}")
y_test = y[sep:]
print(f" Shape of y_test : {y_test.shape}")

 Shape of X_train : (218302, 13)
 Shape of X_test : (72768, 13)
 Shape of y_train : (218302,)
 Shape of y_test : (72768,)


# ML Model

[**Overview CatBoost Regressor**](https://catboost.ai/en/docs/concepts/python-reference_catboostregressor)

In [17]:
Models = CatBoostRegressor(iterations=1200,learning_rate=0.51053,depth=6,loss_function="MAE",
                        eval_metric="MAE",objective='MAE', task_type='GPU',verbose=600,random_state=32,l2_leaf_reg=1.5)

Models.fit(X_train, y_train)

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 505.5620929	total: 113ms	remaining: 2m 15s
600:	learn: 501.4680947	total: 2.87s	remaining: 2.86s
1199:	learn: 499.2981466	total: 5.59s	remaining: 0us


In [18]:
# Make predictions on the testing data
predictions = Models.predict(X_test)

predictions

array([7.55526325, 7.55526325, 3.050041  , ..., 2.99465069, 2.98170811,
       2.75650973])

In [19]:
# calculate MAE 
error = mae(y_test, predictions)

# display 
print("Mean absolute error : " + str(error)) 

Mean absolute error : 128.96434263969203


In [20]:
pred_df = pd.DataFrame(predictions)

pred_df

,0
0,7.555263
1,7.555263
2,3.050041
3,3.050041
4,3.050041
...,...
72763,2.994651
72764,2.999728
72765,2.994651
72766,2.981708


In [21]:
Models = CatBoostRegressor(iterations=1200,learning_rate=0.51053,depth=6,loss_function="MAE",
                        eval_metric="MAE",objective='MAE', task_type='GPU',verbose=600,random_state=32,l2_leaf_reg=1.5)


Models.fit(merged_df_4,y)

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 411.7527742	total: 7.72ms	remaining: 9.26s
600:	learn: 408.3313017	total: 3.27s	remaining: 3.25s
1199:	learn: 406.6351599	total: 6.27s	remaining: 0us


In [22]:
pred = Models.predict(merged_df_4)
pred = pred[-78180:]
pred

array([6.33934949, 6.09566446, 6.09816632, ..., 3.13278916, 3.0424659 ,
       3.03683628])

## Finalization

In [23]:
Sub = pd.read_csv("DataSets/sample_submission.csv")

Sub

,id,net_payment_count
0,202311merchant_36004,0
1,202312merchant_36004,0
2,202310merchant_36004,0
3,202311merchant_23099,0
4,202312merchant_23099,0
...,...,...
78175,202312merchant_35969,0
78176,202310merchant_35969,0
78177,202311merchant_8429,0
78178,202312merchant_8429,0


In [24]:
Sub["net_payment_count"] = pred

Sub = Sub.set_index("id")

Sub

,net_payment_count
id,
202311merchant_36004,6.339349
202312merchant_36004,6.095664
202310merchant_36004,6.098166
202311merchant_23099,6.098166
202312merchant_23099,6.098166
...,...
202312merchant_35969,3.132789
202310merchant_35969,3.001428
202311merchant_8429,3.132789
